In [4]:
## 在猫变虎的融合过程中，可以看出由于两者的“五官”位置差距较大，所以我们需要先找到特征区域，在对图片做仿射变换，使得特征区域可以对上，之后再进行Addweight操作，这样可能最终效果会比较好

### 下面是我参考了github以及论坛资源关于人脸融合的资料，在其基础上理解并改写的人脸融合程序。

import numpy as np
import cv2
import dlib
from scipy.spatial import Delaunay
predictor_model = 'shape_predictor_68_face_landmarks.dat'
## 使用dlib官网提供的提取人脸68个特征点的模型

def get_points(image):  # 用 dlib 来得到人脸的特征点

    face_detector = dlib.get_frontal_face_detector()  # 提取脸外部矩形框
    face_pose_predictor = dlib.shape_predictor(predictor_model) # 根据官网提供的68个人脸特征提取点代码 构造预测器
    detected_face = face_detector(image, 1)[0]
    print(detected_face)
    pose_landmarks = face_pose_predictor(image, detected_face)  # 获取特征
    points = []
    for p in pose_landmarks.parts():
        points.append([p.x, p.y])
    ## 记录特征点的位置
    # 加入四个顶点和四条边的中点
    x = image.shape[1] - 1
    y = image.shape[0] - 1
    points.append([0, 0])
    points.append([x // 2, 0])
    points.append([x, 0])
    points.append([x, y // 2])
    points.append([x, y])
    points.append([x // 2, y])
    points.append([0, y])
    points.append([0, y // 2])

    return np.array(points)


def get_triangles(points):  #  在特征点上使用 Delaunay 三角剖分，将点集连接成一定大小的三角形，且分配要相对合理，才能呈现出漂亮的三角化
    return Delaunay(points).simplices


def affine_transform(input_image, input_triangle, output_triangle, size):  # 这里就是针对三角形进行仿射变化的函数（调用opencv关于仿射变换的库函数），传入的是输入图片，输入特征三角形，输出特征三角形以及输出图片尺寸。对人脸进行仿射变换，确定位置
    warp_matrix = cv2.getAffineTransform(np.float32(input_triangle), np.float32(output_triangle))
    output_image = cv2.warpAffine(input_image, warp_matrix, (size[0], size[1]), None，                  flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)
    return output_image


def morph_triangle(img1, img2, img, tri1, tri2, tri, alpha):  # 三角形变形，Alpha 混合
    # 计算三角形的边界框
    rect1 = cv2.boundingRect(np.float32([tri1]))  # 寻找tri1的左上角坐标，和tri1的长和宽
    rect2 = cv2.boundingRect(np.float32([tri2]))
    rect = cv2.boundingRect(np.float32([tri]))

    tri_rect1 = []
    tri_rect2 = []
    tri_rect_warped = []

    for i in range(0, 3):
        tri_rect_warped.append(
            ((tri[i][0] - rect[0]), (tri[i][1] - rect[1])))
        tri_rect1.append(
            ((tri1[i][0] - rect1[0]), (tri1[i][1] - rect1[1])))
        tri_rect2.append(
            ((tri2[i][0] - rect2[0]), (tri2[i][1] - rect2[1])))

    # 在边界框内进行仿射变换
    img1_rect = img1[rect1[1]:rect1[1] +
                     rect1[3], rect1[0]:rect1[0] + rect1[2]]
    img2_rect = img2[rect2[1]:rect2[1] +
                     rect2[3], rect2[0]:rect2[0] + rect2[2]]

    size = (rect[2], rect[3])
    warped_img1 = affine_transform(
        img1_rect, tri_rect1, tri_rect_warped, size)
    warped_img2 = affine_transform(
        img2_rect, tri_rect2, tri_rect_warped, size)

    # 加权求和
    img_rect = (1.0 - alpha) * warped_img1 + alpha * warped_img2
## 我们在这种三角形区域（包含特征多的区域）进行加权融合，保证融合出来主要是特征区域
    # 生成模板
    mask = np.zeros((rect[3], rect[2], 3), dtype=np.float32)
    cv2.fillConvexPoly(mask, np.int32(tri_rect_warped), (1.0, 1.0, 1.0), 16, 0)

    # 应用模板
    img[rect[1]:rect[1] + rect[3], rect[0]:rect[0] + rect[2]] = \
        img[rect[1]:rect[1] + rect[3], rect[0]:rect[0] +
            rect[2]] * (1 - mask) + img_rect * mask


def morph_faces(filename1, filename2, alpha=0.5):  # 融合图片
    img1 = cv2.imread(filename1)
    img2 = cv2.imread(filename2)
    img2 = cv2.resize(img2,(img1.shape[1],img1.shape[0]),interpolation=cv2.INTER_CUBIC)
    print('img1.shape',img1.shape)
    print('img2.shape',img2.shape)

    points1 = get_points(img1)
    print('pionts1:',len(points1),points1)
    points2 = get_points(img2)
    points = (1 - alpha) * np.array(points1) + alpha * np.array(points2)

    img1 = np.float32(img1)
    img2 = np.float32(img2)
    img_morphed = np.zeros(img1.shape, dtype=img1.dtype)

    triangles = get_triangles(points)
    for i in triangles:
        x = i[0]
        y = i[1]
        z = i[2]

        tri1 = [points1[x], points1[y], points1[z]]
        tri2 = [points2[x], points2[y], points2[z]]
        tri = [points[x], points[y], points[z]]
        morph_triangle(img1, img2, img_morphed, tri1, tri2, tri, alpha)

    return np.uint8(img_morphed)


def main(file1,file2,alpha):
    try:
     alpha = float(alpha)
    except:
        alpha = 0.5
    img_morphed = morph_faces(file1, file2, alpha)
    output_file = '{}_{}_{}.jpg'.format(
        file1.split('.')[0][-2:], file2.split('.')[0][-2:], alpha)
    cv2.imwrite(output_file, img_morphed)
    return output_file

# main('zch.png','zzy.png',0.5)




In [5]:
zzy_img = cv2.imread('zzy.png')
zzy_points = get_points(zzy_img)

[(280, 502) (946, 1168)]


In [6]:
main('pyy.png','zzy.png',0.5)

img1.shape (275, 183, 3)
img2.shape (275, 183, 3)
[(56, 66) (145, 156)]
pionts1: 76 [[ 58  84]
 [ 57  95]
 [ 57 106]
 [ 58 117]
 [ 61 128]
 [ 66 139]
 [ 72 148]
 [ 80 156]
 [ 89 159]
 [ 99 158]
 [108 153]
 [116 145]
 [123 137]
 [128 127]
 [132 117]
 [135 106]
 [137  95]
 [ 67  79]
 [ 74  77]
 [ 82  78]
 [ 89  81]
 [ 95  84]
 [106  84]
 [114  83]
 [121  82]
 [128  83]
 [132  88]
 [100  92]
 [ 99 100]
 [ 98 107]
 [ 97 115]
 [ 89 119]
 [ 92 121]
 [ 96 122]
 [ 99 122]
 [103 121]
 [ 75  88]
 [ 80  87]
 [ 85  88]
 [ 88  92]
 [ 84  92]
 [ 79  91]
 [109  95]
 [113  92]
 [118  92]
 [122  95]
 [118  97]
 [113  97]
 [ 80 132]
 [ 86 131]
 [ 91 130]
 [ 94 132]
 [ 98 131]
 [102 133]
 [106 136]
 [101 138]
 [ 97 139]
 [ 93 139]
 [ 89 138]
 [ 85 135]
 [ 82 133]
 [ 90 134]
 [ 94 135]
 [ 98 135]
 [104 136]
 [ 98 134]
 [ 94 134]
 [ 90 133]
 [  0   0]
 [ 91   0]
 [182   0]
 [182 137]
 [182 274]
 [ 91 274]
 [  0 274]
 [  0 137]]
[(38, 96) (167, 225)]


'yy_zy_0.5.jpg'